# **Loading the dataset**

In [1]:
!wget http://storage.googleapis.com/download.tensorflow.org/data/mini_speech_commands.zip
!unzip -q mini_speech_commands.zip -d .

--2025-12-15 09:24:26--  http://storage.googleapis.com/download.tensorflow.org/data/mini_speech_commands.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.101.207, 142.251.2.207, 74.125.137.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.101.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 182082353 (174M) [application/zip]
Saving to: ‘mini_speech_commands.zip’

mini_speech_command 100%[===================>] 173.65M   204MB/s    in 0.9s    

2025-12-15 09:24:27 (204 MB/s) - ‘mini_speech_commands.zip’ saved [182082353/182082353]



# **Importing the required Libraries**

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt

import librosa
from scipy.fft import fft
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import tensorflow as tf

import glob
import pickle

# **Configuration**

In [3]:
# Configuration
DATA_PATH = "/content/mini_speech_commands"  # The Data path
CLASSES = ['yes', 'no', 'up', 'down', 'left', 'right', 'go', 'stop'] # The 8 target classes

FRAME_LENGTH = 256      # 16ms at 16kHz [ sr=16000 → frame duration = 256/16000 = 0.016 s = 16 ms. ]
HOP_LENGTH = 128        # 8ms step (50% overlap)
SAMPLE_RATE = 16000     # Standard for speech

N_MELS = 40

# **Preprocessing for the audio file**

In [4]:
# Process one WAV file and return its average spectrum
def process_audio_file(file_path):

    # Load audio file
    signal, sr = librosa.load(file_path, sr=SAMPLE_RATE) # Loads audio, resamples to 16 kHz, and converts to mono by default.

    # Divide into frames
    frames = []
    for i in range(0, len(signal) - FRAME_LENGTH, HOP_LENGTH):
        frame = signal[i:i + FRAME_LENGTH]
        frames.append(frame)

    # Apply FFT to each frame and get magnitude spectrum
    spectra = []
    for frame in frames:
        # Apply window function (Hamming) to reduce spectral leakage
        windowed_frame = frame * np.hamming(FRAME_LENGTH)

        # Compute FFT and get magnitude
        spectrum = np.abs(fft(windowed_frame)) # absolute -> magnitude


        # Keep only first half (real FFT symmetry)
        spectrum = spectrum[:FRAME_LENGTH//2]

        spectra.append(spectrum)

    # Average all frames in this file
    if len(spectra) > 0:
        file_average_spectrum = np.mean(spectra, axis=0)
        return file_average_spectrum
    else:
        return None

# **Training Phase**

In [5]:
# Train the distance-based system by creating templates for each class
def train_system(data_path, classes):

    class_templates = {}
    for class_name in classes:
        print(f"Processing class: {class_name}")

        # Get all WAV files in this class folder
        class_path = os.path.join(data_path, class_name)
        wav_files = sorted(glob.glob(os.path.join(class_path, "*.wav")))

        # Process each file in this class
        class_spectra = []
        successful_files = 0

        # Use only 0–799 for training
        train_files = wav_files[:800]

        for wav_file in train_files:
            # Process one audio file
            file_spectrum = process_audio_file(wav_file)

            if file_spectrum is not None:
                class_spectra.append(file_spectrum)
                successful_files += 1

        print(f"  Successfully processed {successful_files}/{len(wav_files)} files")

        # Create class template by averaging all file spectra
        if len(class_spectra) > 0:
            class_template = np.mean(class_spectra, axis=0)
            class_templates[class_name] = class_template
            print(f"  Created template with {len(class_template)} features")
        else:
            print(f"  Warning: No valid files for class {class_name}")

    return class_templates

In [6]:
# Execute training
print("Starting training phase...")
templates = train_system(DATA_PATH, CLASSES)

print(f"\nTraining completed!")
print(f"Created templates for {len(templates)} classes")

Starting training phase...
Processing class: yes
  Successfully processed 800/1000 files
  Created template with 128 features
Processing class: no
  Successfully processed 800/1000 files
  Created template with 128 features
Processing class: up
  Successfully processed 800/1000 files
  Created template with 128 features
Processing class: down
  Successfully processed 800/1000 files
  Created template with 128 features
Processing class: left
  Successfully processed 800/1000 files
  Created template with 128 features
Processing class: right
  Successfully processed 800/1000 files
  Created template with 128 features
Processing class: go
  Successfully processed 800/1000 files
  Created template with 128 features
Processing class: stop
  Successfully processed 800/1000 files
  Created template with 128 features

Training completed!
Created templates for 8 classes


In [7]:
# Save templates for later use
with open('speech_templates.pkl', 'wb') as f:
    pickle.dump(templates, f)

print("Templates saved to 'speech_templates.pkl'")

Templates saved to 'speech_templates.pkl'


# **Testing Phase**

In [8]:
# Load the templates you created in training
with open('speech_templates.pkl', 'rb') as f:
    templates = pickle.load(f)

print("Loaded templates for classes:", list(templates.keys()))

Loaded templates for classes: ['yes', 'no', 'up', 'down', 'left', 'right', 'go', 'stop']


In [9]:
# Compute distance between two vectors
def compute_distance(vector1, vector2, metric='euclidean'):

    if metric == 'euclidean': # L2 distance
        return np.sqrt(np.sum((vector1 - vector2) ** 2))

    elif metric == 'manhattan': # L1 distance
        return np.sum(np.abs(vector1 - vector2))

    elif metric == 'cosine':
        dot_product = np.dot(vector1, vector2)
        norm1 = np.linalg.norm(vector1)
        norm2 = np.linalg.norm(vector2)
        return 1 - (dot_product / (norm1 * norm2))

    else:
        raise ValueError("Unknown metric")

In [10]:
# Predict the class of an unknown speech file
def predict_speech(audio_file_path, templates, metric='euclidean'):

    # Process the test file (same as training)
    test_spectrum = process_audio_file(audio_file_path)

    if test_spectrum is None:
        return None

    # Compute distance to each class template
    distances = {}
    for class_name, template in templates.items():
        distance = compute_distance(test_spectrum, template, metric)
        distances[class_name] = distance

    # Pick class with smallest distance
    predicted_class = min(distances, key=distances.get)

    return predicted_class, distances

In [11]:
# Test the system on all test files and compute accuracy
def test_system(data_path, templates, classes, metric='euclidean'):

    true_labels = []
    predicted_labels = []
    file_paths = []

    print("Starting testing phase...")

    for class_name in classes:
        print(f"Testing class: {class_name}")
        class_path = os.path.join(data_path, class_name)

        # Load and sort wav files
        wav_files = sorted(glob.glob(os.path.join(class_path, "*.wav")))

        # files 0–799 train, 800–999 test
        test_files = wav_files[800:1000]

        for test_file in test_files:
            # Get prediction
            prediction, distances = predict_speech(test_file, templates, metric)

            if prediction is not None:
                true_labels.append(class_name)
                predicted_labels.append(prediction)
                file_paths.append(test_file)

                # Optional: Print some predictions
                if len(true_labels) % 100 == 0:
                    print(f"  Sample: {os.path.basename(test_file)}")
                    print(f"    True: {class_name}, Predicted: {prediction}")
                    print(f"    Distances: {dict(list(distances.items())[:2])}...")

    return true_labels, predicted_labels, file_paths

**Evaluation**

In [12]:
#Compute and display evaluation metrics
def evaluate_performance(true_labels, predicted_labels, classes):

    # Overall accuracy
    accuracy = accuracy_score(true_labels, predicted_labels)
    print(f"\n{'='*50}")
    print(f"📊 FINAL RESULTS")
    print(f"{'='*50}")
    print(f"Overall Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")

    # Per-class accuracy
    print(f"\nPer-class Accuracy:")
    cm = confusion_matrix(true_labels, predicted_labels, labels=classes)

    for i, class_name in enumerate(classes):
        class_correct = cm[i, i]
        class_total = np.sum(cm[i, :])
        class_accuracy = class_correct / class_total if class_total > 0 else 0
        print(f"  {class_name:6}: {class_accuracy:.4f} ({class_correct}/{class_total})")

    # Confusion matrix
    print(f"\nConfusion Matrix:")
    print("True \\ Pred", end="")
    for class_name in classes:
        print(f"{class_name:8}", end="")
    print()

    for i, true_class in enumerate(classes):
        print(f"{true_class:11}", end="")
        for j in range(len(classes)):
            print(f"{cm[i, j]:8}", end="")
        print()

    return accuracy, cm

In [13]:
print("=== Testing Distance-Based Speech Recognition ===")

# Load templates
with open('speech_templates.pkl', 'rb') as f:
  templates = pickle.load(f)

print(f"Loaded templates for {len(templates)} classes")

# Test with different distance metrics
metrics = ['euclidean', 'manhattan', 'cosine']

best_accuracy = 0
best_metric = None

for metric in metrics:
  print(f"\n{'='*40}")
  print(f"Testing with {metric} distance:")
  print(f"{'='*40}")

  true_labels, predicted_labels, file_paths = test_system(DATA_PATH, templates, CLASSES, metric=metric)

  accuracy, cm = evaluate_performance(true_labels, predicted_labels, CLASSES)

  if accuracy > best_accuracy:
    best_accuracy = accuracy
    best_metric = metric

print(f"\n🎯 BEST PERFORMANCE: {best_accuracy:.4f} with {best_metric} distance")

# Save results
results = {
  'true_labels': true_labels,
  'predicted_labels': predicted_labels,
  'file_paths': file_paths,
  'accuracy': best_accuracy,
  'best_metric': best_metric
  }

with open('test_results.pkl', 'wb') as f:
  pickle.dump(results, f)

print("✅ Test results saved to 'test_results.pkl'")

=== Testing Distance-Based Speech Recognition ===
Loaded templates for 8 classes

Testing with euclidean distance:
Starting testing phase...
Testing class: yes
  Sample: e7d0eb3f_nohash_0.wav
    True: yes, Predicted: yes
    Distances: {'yes': np.float64(4.03366885434509), 'no': np.float64(4.772526170469457)}...
  Sample: ffd2ba2f_nohash_2.wav
    True: yes, Predicted: yes
    Distances: {'yes': np.float64(1.36645282084867), 'no': np.float64(2.239572092379814)}...
Testing class: no
  Sample: e53139ad_nohash_4.wav
    True: no, Predicted: go
    Distances: {'yes': np.float64(1.5416231991706757), 'no': np.float64(1.5446076385167504)}...
  Sample: fffcabd1_nohash_0.wav
    True: no, Predicted: yes
    Distances: {'yes': np.float64(1.1503871560846637), 'no': np.float64(1.7202164286991537)}...
Testing class: up
  Sample: e62056e2_nohash_0.wav
    True: up, Predicted: stop
    Distances: {'yes': np.float64(6.203250851550998), 'no': np.float64(6.4487657083937995)}...
  Sample: ffd2ba2f_nohas

# **CNN**

## Load WAV → Mel Spectrogram

In [14]:
def extract_mel_spectrogram(file_path):
    signal, sr = librosa.load(file_path, sr=SAMPLE_RATE)

    # Compute Mel spectrogram
    mel = librosa.feature.melspectrogram(
        y=signal,
        sr=SAMPLE_RATE,
        n_fft=512,
        hop_length=160,
        n_mels=N_MELS
    )

    # Convert to log scale
    mel_db = librosa.power_to_db(mel, ref=np.max)

    # Normalize
    mel_db = (mel_db - mel_db.min()) / (mel_db.max() - mel_db.min())

    return mel_db.T   # Shape: (time, mel)

## Load the dataset

In [15]:
def load_dataset():
    X = []
    y = []

    for label_idx, class_name in enumerate(CLASSES):
        class_path = os.path.join(DATA_PATH, class_name)
        wav_files = sorted([f for f in os.listdir(class_path) if f.endswith(".wav")])

        for file in wav_files:
            mel = extract_mel_spectrogram(os.path.join(class_path, file))
            X.append(mel)
            y.append(label_idx)

    return X, np.array(y)

## Pad sequences

In [16]:
def pad_spectrograms(X):
    max_len = max([x.shape[0] for x in X])
    padded = []

    for m in X:
        pad_width = max_len - m.shape[0]
        padded.append(np.pad(m, ((0, pad_width), (0,0)), mode='constant'))

    return np.array(padded)

## Prepare data for training

In [17]:
X, y = load_dataset()
X = pad_spectrograms(X)

# Add channel dimension for CNN
X = X[..., np.newaxis]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

## Build the CNN model

In [18]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', padding='same'),
    tf.keras.layers.MaxPool2D((2,2)),

    tf.keras.layers.Conv2D(32, (3,3), activation='relu', padding='same'),
    tf.keras.layers.MaxPool2D((2,2)),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    tf.keras.layers.MaxPool2D((2,2)),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(len(CLASSES), activation='softmax')
])

model.compile(
    optimizer="Adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

## Train the model

In [19]:
history = model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=32,
    validation_split=0.2
)

Epoch 1/30
160/160 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.2573 - loss: 1.9015 - val_accuracy: 0.6180 - val_loss: 1.0969
Epoch 2/30
160/160 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6299 - loss: 1.0684 - val_accuracy: 0.7984 - val_loss: 0.6617
Epoch 3/30
160/160 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7552 - loss: 0.7034 - val_accuracy: 0.8375 - val_loss: 0.5228
Epoch 4/30
160/160 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8210 - loss: 0.5234 - val_accuracy: 0.8609 - val_loss: 0.4312
Epoch 5/30
160/160 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8488 - loss: 0.4276 - val_accuracy: 0.8617 - val_loss: 0.4087
Epoch 6/30
160/160 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8854 - loss: 0.3385 - val_accuracy: 0.8805 - val_loss: 0.3607
Epoch 7/30
160/160 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9010 - loss: 0.2830 - val_accuracy: 0.8828 - val_loss: 0.3535
Epoch 8/30
160/160 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9163 - loss: 0.2364 - val_accuracy: 0.

## Evaluate

In [20]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc*100:.2f}%")

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9044 - loss: 0.5731
Test Accuracy: 90.81%


## Predict a single WAV

In [21]:
def predict_file(path):
    mel = extract_mel_spectrogram(path)
    mel = np.pad(mel, ((0, X.shape[1] - mel.shape[0]), (0, 0)))
    mel = mel[np.newaxis, ..., np.newaxis]
    probs = model.predict(mel)[0]
    pred = np.argmax(probs)
    return CLASSES[pred], probs[pred]